In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
# Load th dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the data
## Drop irrelevant features
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data.Geography.value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [5]:
data.Gender.value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [6]:
label_encoder_gender = LabelEncoder()
data.Gender = label_encoder_gender.fit_transform(data['Gender'])

In [7]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
data.Gender.value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [9]:
#one hot encoder
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [10]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
#combine colums
data = pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
# save the encoder and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [14]:
#divide date into input(independent) and output(dependent)
X = data.drop('Exited',axis=1)
y = data.Exited

In [15]:
#train, test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=100)

In [16]:
#scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [17]:
X_train

array([[ 0.35476836,  0.9111777 , -0.18423398, ..., -0.99925028,
         1.72286214, -0.57484841],
       [ 0.30282874, -1.09748077, -0.2796117 , ...,  1.00075028,
        -0.58042949, -0.57484841],
       [ 0.02235481, -1.09748077, -0.08885627, ..., -0.99925028,
        -0.58042949,  1.73958906],
       ...,
       [ 0.23011328, -1.09748077,  0.38803232, ...,  1.00075028,
        -0.58042949, -0.57484841],
       [-0.60092059, -1.09748077, -0.75650029, ...,  1.00075028,
        -0.58042949, -0.57484841],
       [-1.79553176, -1.09748077, -1.42414431, ..., -0.99925028,
         1.72286214, -0.57484841]])

In [18]:
X_test

array([[-0.21659712,  0.91739884, -1.51293681, ..., -1.01714699,
         1.74836928, -0.56965192],
       [-0.6038873 , -1.09003844, -0.65561389, ..., -1.01714699,
         1.74836928, -0.56965192],
       [ 0.34395444,  0.91739884, -0.56035579, ..., -1.01714699,
         1.74836928, -0.56965192],
       ...,
       [ 0.20126859,  0.91739884, -0.17932338, ...,  0.98314207,
        -0.57196155, -0.56965192],
       [-0.02295204,  0.91739884, -1.2271625 , ..., -1.01714699,
         1.74836928, -0.56965192],
       [ 2.00522544,  0.91739884, -0.27458148, ...,  0.98314207,
        -0.57196155, -0.56965192]])

In [19]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Implementation

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [21]:
X_train.shape[1]

12

In [22]:
(X_train.shape[1],)

(12,)

In [23]:
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), # Hidden layer 1 connecte with input layer
    Dense(32,activation='relu'), # Hidden layer 2
    Dense(1,activation='sigmoid') # output layer
])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
import tensorflow
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [26]:
#compile the model
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [27]:
#set up tensorboard
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [28]:
# set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [29]:
#train the model
history= model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 7s 11ms/step - loss: 0.3934 - accuracy: 0.8332 - val_loss: 0.3779 - val_accuracy: 0.8500
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3527 - accuracy: 0.8566 - val_loss: 0.3711 - val_accuracy: 0.8535
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3417 - accuracy: 0.8624 - val_loss: 0.3715 - val_accuracy: 0.8460
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3405 - accuracy: 0.8602 - val_loss: 0.3786 - val_accuracy: 0.8450
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3334 - accuracy: 0.8641 - val_loss: 0.3767 - val_accuracy: 0.8520
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3319 - accuracy: 0.8626 - val_loss: 0.3750 - val_accuracy: 0.8500
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3298 - accuracy: 0.8656 - val_loss: 0.3718 - val_accuracy: 0.8

In [30]:
model.save('model.h5')

e:\Udemy_Projects\Ann_project\ANN_Classifiation_mywork\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
#load tensorboard extensionn
%load_ext tensorboard

In [32]:
%tensorboard --logdir logs/fit


Reusing TensorBoard on port 6007 (pid 5584), started 2:10:28 ago. (Use '!kill 5584' to kill it.)